In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
# import cv2
from pathlib import Path
from PIL import Image
import os
import torchvision.transforms as transforms

In [50]:
# keypoints = pd.read_csv("train_labels.csv", index_col="file_name")

# file_names = os.listdir('train')

# for file_name in file_names:
#     if keypoints.loc[file_name].shape != (18,):
#         print(file_name)

In [51]:
# for i in keypoints.columns:
#     print(f"\"{i}\",")

In [52]:
keypoints = pd.read_csv("data/train_labels.csv")
filtered_data = keypoints[keypoints['file_name'] == 'e4f6e24a432559c091e0292b12024466.jpg']
filtered_data

,left_eye_x,left_eye_y,right_eye_x,right_eye_y,mouth_x,mouth_y,left_ear_1_x,left_ear_1_y,left_ear_2_x,left_ear_2_y,left_ear_3_x,left_ear_3_y,right_ear_1_x,right_ear_1_y,right_ear_2_x,right_ear_2_y,right_ear_3_x,right_ear_3_y,file_name
448,406.0,330.0,609.0,358.0,486.0,485.0,305.0,259.0,237.0,37.0,385.0,202.0,655.0,241.0,810.0,124.0,733.0,320.0,e4f6e24a432559c091e0292b12024466.jpg
1968,207.0,216.0,292.0,219.0,246.0,292.0,155.0,167.0,153.0,65.0,216.0,136.0,292.0,137.0,366.0,81.0,348.0,179.0,e4f6e24a432559c091e0292b12024466.jpg


In [53]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from PIL import Image
# import numpy as np

# # Замените путь на реальный путь к вашему изображению
# image_path = 'train/7d3cdffd19b9cec0d44661797492b94a.jpg'

# # Открываем изображение
# image = Image.open(image_path)

# # Создаем объект для рисования
# fig, ax = plt.subplots()

# # Отображаем изображение
# ax.imshow(image)

# # Пример набора точек
# points1 = keypoints[keypoints['file_name'] == '7d3cdffd19b9cec0d44661797492b94a.jpg'].loc[140].values[:-1].astype(float)
# points2 = keypoints[keypoints['file_name'] == '7d3cdffd19b9cec0d44661797492b94a.jpg'].loc[1649].values[:-1].astype(float)

# # Рисуем точки на изображении
# for i in range(0, len(points1), 2):
#     x, y = points1[i], points1[i + 1]
#     circle = patches.Circle((x, y), radius=5, color='red')
#     ax.add_patch(circle)

# for i in range(0, len(points2), 2):
#     x, y = points2[i], points2[i + 1]
#     circle = patches.Circle((x, y), radius=5, color='blue')
#     ax.add_patch(circle)

# plt.show()

In [54]:
# # Замените путь на реальный путь к вашему изображению
# image_path = 'train/e4f6e24a432559c091e0292b12024466.jpg'

# # Открываем изображение
# image = Image.open(image_path)

# # Создаем объект для рисования
# fig, ax = plt.subplots()

# # Отображаем изображение
# ax.imshow(image)

# # Пример набора точек
# points1 = keypoints[keypoints['file_name'] == 'e4f6e24a432559c091e0292b12024466.jpg'].loc[448].values[:-1].astype(float)
# points2 = keypoints[keypoints['file_name'] == 'e4f6e24a432559c091e0292b12024466.jpg'].loc[1968].values[:-1].astype(float)

# # Рисуем точки на изображении
# for i in range(0, len(points1), 2):
#     x, y = points1[i], points1[i + 1]
#     circle = patches.Circle((x, y), radius=5, color='red')
#     ax.add_patch(circle)

# for i in range(0, len(points2), 2):
#     x, y = points2[i], points2[i + 1]
#     circle = patches.Circle((x, y), radius=5, color='blue')
#     ax.add_patch(circle)

# plt.show()

In [55]:
# keypoints.loc['7d3cdffd19b9cec0d44661797492b94a.jpg']

In [56]:
# keypoints.loc['e4f6e24a432559c091e0292b12024466.jpg']

In [57]:
keypoints = keypoints.drop(index=140)
keypoints = keypoints.drop(index=448)
keypoints = keypoints.set_index('file_name')

In [58]:
class CatsDataset(Dataset):
    def __init__(self, folder, keypoint_data):
        self.folder = folder
        self.keypoint_data = keypoint_data
        self.image_paths = [os.path.join(folder, filename) for filename in os.listdir(folder)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        image = image.resize((256, 256))
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
        ])
        image = transform(image)

        file_name = os.path.basename(image_path)
        keypoints = torch.tensor(self.keypoint_data.loc[file_name].values)
        return image.float(), keypoints.float()

In [59]:
class TestDataset(Dataset):
    def __init__(self, folder):
        self.folder = folder
        self.image_paths = [os.path.join(folder, filename) for filename in os.listdir(folder)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        image = image.resize((256, 256))
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        image = transform(image)
        file_name = os.path.basename(image_path)
        return image.float(), file_name

In [60]:
# class KeypointCNN(nn.Module):
#     def __init__(self):
#         super(KeypointCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
#         self.fc1 = nn.Linear(64 * 64 * 4, 128)
#         self.fc2 = nn.Linear(128, 18)

#     def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = x.view(-1, 64 * 64 * 4)
#         x = nn.functional.relu(self.fc1(x))
#         x = nn.functional.relu(self.fc2(x))
#         return x

In [61]:
# class KeypointCNN(nn.Module):
#     def __init__(self):
#         super(KeypointCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.fc1 = nn.Linear(256 * 16 * 16, 1024)
#         self.fc2 = nn.Linear(1024, 512)
#         self.fc3 = nn.Linear(512, 256)
#         self.fc4 = nn.Linear(256, 128)
#         self.fc5 = nn.Linear(128, 18)
#
#     def forward(self, x):
#         x = self.pool(torch.nn.functional.relu(self.conv1(x)))
#         x = self.pool(torch.nn.functional.relu(self.conv2(x)))
#         x = self.pool(torch.nn.functional.relu(self.conv3(x)))
#         x = self.pool(torch.nn.functional.relu(self.conv4(x)))
#         x = x.view(-1, 256 * 16 * 16)
#         x = torch.nn.functional.relu(self.fc1(x))
#         x = torch.nn.functional.relu(self.fc2(x))
#         x = torch.nn.functional.relu(self.fc3(x))
#         x = torch.nn.functional.relu(self.fc4(x))
#         x = self.fc5(x)
#         return x


class KeypointCNN(nn.Module):
    def __init__(self):
        super(KeypointCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 512)
        self.fc2 = nn.Linear(512, 18)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [62]:
#keypoints = pd.read_csv("train_labels.csv", index_col="file_name")
train_dataset = CatsDataset("data/images/train/", keypoints)
test_dataset = TestDataset("data/images/test/")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
from torch.optim import lr_scheduler
num_epochs = 2
learning_rate = 0.001

model = KeypointCNN()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

# scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
#
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

for epoch in range(num_epochs):
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
torch.save(model, f"./{10}_epochs_big.pth")

In [ ]:
import csv

model.eval()

predictions = []

with torch.no_grad():
    for images, image_paths in test_loader: 
        outputs = model(images)
        predicted_keypoints = outputs.detach().numpy() 

        # Add 'file_name' column to predictions
        predictions_with_filenames = [{"file_names": path, **{f"{key}": value for key, value in zip(["left_eye_x", "left_eye_y", "right_eye_x", "right_eye_y", "mouth_x", "mouth_y", "left_ear_1_x", "left_ear_1_y", "left_ear_2_x", "left_ear_2_y", "left_ear_3_x", "left_ear_3_y", "right_ear_1_x", "right_ear_1_y", "right_ear_2_x", "right_ear_2_y", "right_ear_3_x", "right_ear_3_y"], keypoints)}} for path, keypoints in zip(image_paths, predicted_keypoints)]
        predictions.extend(predictions_with_filenames)

# Write predictions to CSV
csv_columns = ["file_names", "left_eye_x", "left_eye_y", "right_eye_x", "right_eye_y", "mouth_x", "mouth_y", "left_ear_1_x", "left_ear_1_y", "left_ear_2_x", "left_ear_2_y", "left_ear_3_x", "left_ear_3_y", "right_ear_1_x", "right_ear_1_y", "right_ear_2_x", "right_ear_2_y", "right_ear_3_x", "right_ear_3_y"]
with open("test_predictions.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(predictions)
